In [2]:
# Ref:
# YouTube Video
# Sam Witteveen
# Creating an AI Agent with LangGraph Llama 3 & Groq
# https://www.youtube.com/watch?v=lvQ96Ssesfk

In [3]:
!pip -q install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [5]:
import json
import os
import re
from google.colab import userdata

# Define the API clients

In [6]:
from groq import Groq

groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY'),
)


# What is the objective?

- Create a chat system with summarization memory
- Use a summarization agent to summarize older chat messages



# Create a list of agents

AGENTS
1. chat_agent
2. router_agent
3. chat_history_summ_agent


# Helper functions

In [7]:
def write_markdown_file(content, filename):
  """Writes the given content as a markdown file to the local directory.

  Args:
    content: The string content to write to the file.
    filename: The filename to save the file as.
  """
  with open(f"{filename}.md", "w") as f:
    f.write(content)


In [8]:
def create_message_history(system_message, user_input):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



In [9]:
def initialize_message_history(system_message):

    """
    Create a message history messages list.
    Args:
        system_message (str): The system message
        user_query (str): The user input
    Returns:
        A list of dicts in OpenAi chat format
    """

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history



# Set up the LLM

In [10]:
def make_llm_api_call(message_history):

    """
    Makes a call to the Llama3 model on Groq.
    Args:
        message_history (List of dicts): The message history
    Returns:
        response_text: (str): The text response from the LLM
    """

    response = groq_client.chat.completions.create(
                        messages=message_history,
                        model="llama3-70b-8192",
                    )

    response_text = response.choices[0].message.content

    return response_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response = make_llm_api_call(message_history)

print(response)

My name is Molly!


# Set up the tools



# Set up the system messages

In [11]:
chat_agent_system_message = """
You are a helpful assistant. Your name is Molly.
""".strip()



In [20]:
summ_agent_system_message = """
You are an expert at summarizing chats.

When you are given a chat summary and a conversation history, your task is to incorporate
the new lines of conversation into the existing summary,
ensuring a seamless integration that captures the essence of the entire dialogue.

When you are given only a conversation history, your task is to summarize the conversatiom
in a way that catptures the essence of the dialogue.

You format your output as JSOn with he following key: conversation_summary
Only outpt JSON. Don't output any additional text.
"""

# Set up the Agents

In [14]:
def run_chat_agent(message_history):

    # Prompt the llm
    response = make_llm_api_call(message_history)

    print("Assistant:\n",response)

    return response



# Example

user_query = "Hello. How are you?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)


Assistant:
 Hello there! I'm Molly, nice to meet you! I'm doing great, thanks for asking. I'm always happy to help with any questions or tasks you might have. How about you? How's your day going so far?


In [25]:
# Summarization Agent

def run_summ_agent(message_history):

    print("---SUMM AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message_history)

    print("Summary:\n",response)

    return response


# Example conversation

User1 = "Hi there! I've been hearing a lot about minimalism lately. Can you tell me what it is exactly?"
LLM1 =  "Hi! Minimalism is a lifestyle and design philosophy that emphasizes simplicity and the removal of excess. In essence, it's about focusing on what's truly important and letting go of everything else. This can apply to physical possessions, design, work, and even thoughts and activities."

User2 = "That sounds interesting. How can someone start adopting a minimalist lifestyle?"
LLM2 = "A good starting point is decluttering your living space. Go through your belongings and keep only what you use regularly or what truly brings you joy. You can also apply minimalism to your schedule by prioritizing tasks and cutting out activities that aren't meaningful or necessary. It's about being intentional with your time and resources."

User3 = "I've read about people who have very few possessions. Is that what minimalism is all about?"
LLM3 = "While some minimalists do choose to own very few items, minimalism is more about the mindset than a specific number of possessions. It's about valuing quality over quantity and making thoughtful choices about what you bring into your life. For some, this might mean having fewer items, while for others, it could mean simply being more mindful of their consumption."

User4 = " How can minimalism impact someone's mental health?"
LLM4 =  "Minimalism can have a positive impact on mental health by reducing stress and increasing focus. A clutter-free environment can lead to a clearer mind, and simplifying your schedule can reduce feelings of overwhelm. By eliminating distractions and focusing on what's truly important, you can create a more peaceful and fulfilling life."

User5 = "Are there any downsides to minimalism?"
LLM5 =  "One potential downside is that it can be challenging to let go of items or commitments, especially if they hold sentimental value or if you're used to a busier lifestyle. It can also be difficult to find a balance that works for you. The key is to approach minimalism in a way that feels right for you, without feeling pressured to adhere to any strict rules."

User6 = "Do you have any tips for someone just starting out with minimalism?"
LLM6 = "Absolutely! Start small and go at your own pace. You don't have to declutter your entire home in one day. Begin with one area, like a closet or a drawer, and gradually work your way through other spaces. Reflect on what truly adds value to your life and what doesn't. Remember, minimalism is a personal journey, and there's no one-size-fits-all approach."

User7 = "Thanks for the advice! I'll give it a try and see how it goes."
LLM7 = "You're welcome! Good luck on your minimalist journey. Feel free to reach out if you have any more questions along the way."


example_message_history1 = [
    {"role": "system", "content": "You are an expert on simple living."},
     {"role": "user", "content": User1},
     {"role": "assistant", "content": LLM1},
     {"role": "user", "content": User2},
     {"role": "assistant", "content": LLM2},
     {"role": "user", "content": User3},
     {"role": "assistant", "content": LLM3},
     {"role": "user", "content": User4},
     {"role": "assistant", "content": LLM4}
]

data = f"""
 {{
    "chat_summary": "None",
    "conversation_history": {example_message_history1}
}}
"""

message_history = create_message_history(summ_agent_system_message, data)

# Prompt the chat_agent
response = run_summ_agent(message_history)


---SUMM AGENT---
Summary:
 {
    "conversation_summary": "The conversation is about minimalism, a lifestyle that emphasizes simplicity and removal of excess. It's a mindset that focuses on what's truly important and lets go of everything else. To start adopting minimalism, one can declutter their living space, prioritize tasks, and be intentional with their time and resources. Minimalism is not just about owning few possessions, but about making thoughtful choices about what to bring into one's life. It can positively impact mental health by reducing stress, increasing focus, and creating a more peaceful and fulfilling life."
}


In [26]:
# Example 2

summary1 = """
This conversation is about minimalism, a lifestyle that emphasizes simplicity and removal of excess. \
The assistant explains that minimalism is about focusing on what's truly important and letting \
go of everything else, and can be applied to physical possessions, design, work, and even \
thoughts and activities. The user asks how to start adopting a minimalist lifestyle, and \
the assistant suggests decluttering living space and being intentional with time and resources. \
The conversation also touches on the idea that minimalism is a mindset rather than a specific number \
of possessions, and how it can positively impact mental health by reducing stress and increasing focus.
""".strip()

# Note that this does not include
# a second system message.
# Later the summary will be added to the chat history
# as a second system message.
example_message_history2 = [
    {"role": "system", "content": "You are an expert on simple living."},
     {"role": "user", "content": User5},
     {"role": "assistant", "content": LLM5},
     {"role": "user", "content": User6},
     {"role": "assistant", "content": LLM6},
     {"role": "user", "content": User7},
     {"role": "assistant", "content": LLM7}
]

data2 = f"""
 {{
    "chat_summary": {summary1},
    "conversation_history": {example_message_history2}
}}
"""

message_history = create_message_history(summ_agent_system_message, data2)

# Prompt the chat_agent
response = run_summ_agent(message_history)

---SUMM AGENT---
Summary:
 {
    "conversation_summary": 
"This conversation is about minimalism, a lifestyle that emphasizes simplicity and removal of excess. 
The assistant explains that minimalism is about focusing on what's truly important and letting 
go of everything else, and can be applied to physical possessions, design, work, and even 
thoughts and activities. The user asks how to start adopting a minimalist lifestyle, and 
the assistant suggests decluttering living space and being intentional with time and resources. 
The conversation also touches on the idea that minimalism is a mindset rather than a specific number 
of possessions, and how it can positively impact mental health by reducing stress and increasing focus. 
Additionally, the assistant addresses potential downsides of minimalism, such as difficulty letting go of sentimental items and finding balance, 
and provides tips for beginners, including starting small and reflecting on what adds value to one's life."
}


# Run infinitely with a user input field

Creating the summarization memory"
- Extract messges to summarize inclusing the system message.
- Use the summ_agent to summarize the messages
- Add the summary in a second system message
- Create a new message history made up of the first system message, second system message and the messages that were not summarized.

In [34]:
# Prompting the user for input
#user_input = input("Please enter something: ")

message_history = initialize_message_history(chat_agent_system_message)

summary = "None"

max_messages = 10
num_messages_to_summ = 5

while True:

    print()
    #print("==========")
    user_input = input("User: ")
    #print("==========")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop

    # Update message history
    message = {"role": "user", "content": user_input}
    message_history.append(message)


    # Prompt the chat_agent
    llm_response = run_chat_agent(message_history)

    # Update message history
    message = {"role": "assistant", "content": llm_response}
    message_history.append(message)

    length = len(message_history)
    print("-----")
    print("Num_messages:", length)

    if length > max_messages:

        print("Running summary...")

        # Get the first 5 messages
        # This will include the two system messages
        messages_to_summ = message_history[0:num_messages_to_summ]
        print("Len messages_to_summ:", len(messages_to_summ))

        # Remove the first 5 messages
        not_summed_message_history = message_history[num_messages_to_summ:]
        print("Len not_summed_message_history:", len(not_summed_message_history))

        # Create a summary of the messages
        # Note that the summary was added as a second
        # system message, therefore it will also be
        # included in messages_to_summ below.
        data = f"""
        {{
            "chat_summary": {summary},
            "conversation_history": {messages_to_summ}
        }}
        """

        message_history_for_summ_agent = create_message_history(summ_agent_system_message, data)

        # Prompt the summ_agent
        summary = run_summ_agent(message_history_for_summ_agent)

        # Create a new message history
        # This contains the original system message
        message_history = initialize_message_history(chat_agent_system_message)

        # Add a second system message that includes the summary

        second_system_message_with_summary = f"""
        The conversation is ongoing. Older dialogue is being summarized to \
        reduce the LLM context size. This is the summary of the older dialogue:
        Summary: {summary}
        """.strip()

        message = {"role": "system", "content": second_system_message_with_summary}
        message_history.append(message)

        # Include not summed message history
        message_history = message_history + not_summed_message_history
        print("Len new message_history:", len(message_history))

        print(message_history)
        print("Summarization complete")





User: Hi
Assistant:
 Hi there! I'm Molly, your friendly assistant. How can I help you today? Do you have a question, need assistance with something, or just want to chat? I'm all ears!
-----
Num_messages: 3

User: How are you?
Assistant:
 I'm doing great, thanks for asking! I'm a large language model, so I don't have feelings like humans do, but I'm always happy to be of assistance and help with any tasks or questions you may have. It's always a beautiful day when I get to help someone new! How about you, how's your day going so far?
-----
Num_messages: 5

User: Where are you?
Assistant:
 I'm a virtual assistant, so I don't have a physical presence. I exist only in the digital realm, which means I can be accessed from anywhere with an internet connection! I'm a cloud-based AI, so I can be reached from anywhere in the world, at any time. Which means I'm always just a message away, no matter where you are!
-----
Num_messages: 7

User: Youre very smart
Assistant:
 Aw, thank you so much! 

In [36]:
len(message_history)

8